In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
oct_data_ = pd.read_csv('/kaggle/input/ecommerce-events-history-in-cosmetics-shop/2019-Oct.csv')


In [ ]:
oct_data_.info(null_counts=True)


In [ ]:
oct_data_['event_time'] = pd.to_datetime(oct_data_['event_time'],infer_datetime_format=True)
oct_data_.info(null_counts=True)

In [ ]:
oct_data_.head(5)
oct_data_.shape

In [ ]:
oct_data_.loc[oct_data_['brand'].isnull(),'brand']='Non Brand'

In [ ]:
oct_data_[oct_data_['price']<0]

<h3>the negetive prices are on the items that were puchased, so it seems like that negetive price represents the item return</h3>

In [ ]:
oct_data_.head(10)

In [ ]:
#Lets see a pie chart of event_type

sizes_ = oct_data_.groupby(['event_type']).size().reset_index(name='Count')
print(sizes_)

labels_ = sizes_['event_type'].to_list()
print(labels_)

counter_ = sizes_['Count'].to_list()
print(counter_)
total = sum([int(v) for v in counter_])
print(total)
perchange = [(v/total)*100 for v in counter_]
print(perchange)
fig, ax1 = plt.subplots()
ax1.pie(perchange,labels=labels_,autopct='%1.1f%%',shadow=True, startangle=90)
ax1.axis('equal')
plt.show()

In [ ]:
customer_data = oct_data_[['user_id','event_type','price']]

In [ ]:
price_df_ = customer_data[['user_id','price']]
price_df_= price_df_.groupby(['user_id'])['price'].sum().reset_index()
price_df_= price_df_.sort_values(by=['price'],ascending=False)
price_df_ = price_df_.rename(columns={'price':'spent'})
print(price_df_.shape)

In [ ]:
print(price_df_['user_id'].describe())
print(price_df_['spent'].describe())


In [ ]:
price_lst_ = price_df_['spent'].to_list()
plt.hist(x= price_lst_,bins=100,log=True, color='red')
plt.xlabel('Spent amount')
plt.ylabel('Customer Count')
plt.title('Customer Spending in October')
plt.show()

In [ ]:
sell_df_ = oct_data_[['event_time','event_type']]
sell_df_.shape

In [ ]:
sell_df_ = sell_df_[sell_df_['event_type']=='purchase']
sell_df_.shape

In [ ]:
sell_df_['event_time'] = sell_df_['event_time'].dt.weekday

In [ ]:
sell_df_ = sell_df_.groupby(['event_time']).size().reset_index(name='Count')

In [ ]:

days = ['Mon','Tue','Wed','Thu','Fri','Sat','Sun']
day_id = sell_df_['event_time'].to_list()
sold_count = sell_df_['Count'].to_list()

print(days)
print(day_id)
print(sold_count)

sun_sold = sold_count[6]
days = ['Sun']+days[:-1]
sold_count=[sun_sold]+sold_count[:-1]



In [ ]:
plt.bar(day_id,sold_count)
plt.xlabel('Day of Week')
plt.ylabel('Total Sell count')
plt.title('Total sell in each weekday in October')
plt.xticks(day_id,days)
plt.show()

In [ ]:
customer_info_ = oct_data_[['user_id','event_type']]
unq = list(customer_info_['user_id'].unique())
print('unique_customer ',len(unq))


In [ ]:
view_df = customer_info_[customer_info_['event_type']=='view']
view_df = view_df.groupby(['user_id']).size().reset_index(name='count')
view_dict={x:y for x,y in zip(view_df['user_id'].to_list(),view_df['count'].to_list())}

len(view_dict)


In [ ]:
cart_df = oct_data_[oct_data_['event_type']=='cart']
cart_df = cart_df.groupby(['user_id']).size().reset_index(name='count')
cart_dict={x:y for x,y in zip(cart_df['user_id'].to_list(),cart_df['count'].to_list())}
len(cart_dict)


In [ ]:
purchase_df = customer_info_[customer_info_['event_type']=='purchase']
purchase_df = purchase_df.groupby(['user_id']).size().reset_index(name='count')
purchase_dict={x:y for x,y in zip(purchase_df['user_id'].to_list(),purchase_df['count'].to_list())}

len(purchase_dict)


In [ ]:
v = []
c = []
p = []

n_v = []
n_c = []
n_p = []

i = 0 
j = 0
for id_ in unq:
    if purchase_dict.get(id_,0)>0:
        i+=1
        v.append(view_dict.get(id_,0))
        c.append(cart_dict.get(id_,0))
        p.append(purchase_dict.get(id_,0))
    else:
        j+=1
        n_v.append(view_dict.get(id_,0))
        n_c.append(cart_dict.get(id_,0))
        n_p.append(purchase_dict.get(id_,0))

print(i,' users at least purchased once')
print(j, ' users never did any purchase')

In [ ]:
fig, ax1 = plt.subplots()
ax1.scatter(v,p,c='g',marker='.')
ax1.scatter(n_v,n_p,c='r',marker='x')
ax1.set_xlabel('view')
ax1.set_ylabel('purchase')
plt.title('view purchase graph')
plt.show()

In [ ]:
fig, ax1 = plt.subplots()
ax1.scatter(c,p,c='g',marker='.')
ax1.scatter(n_c,n_p,c='r',marker='x')
ax1.set_xlabel('cart')
ax1.set_ylabel('purchase')
plt.title('cart purchase graph')
plt.show()

In [ ]:
fig, ax1 = plt.subplots()
ax1.scatter(v,c,c='g',marker='.')
ax1.scatter(n_v,n_c,c='r',marker='x')
ax1.set_xlabel('view')
ax1.set_ylabel('cart')
plt.title('view cart graph')
plt.show()

In [ ]:
time_series_df = oct_data_[['event_time','event_type']]
time_series_df['event_time'] = time_series_df['event_time'].dt.date

In [ ]:
sell_by_day = time_series_df.groupby('event_time').size().reset_index(name='Count')
plt.figure(figsize=(40,20))
plt.xticks(rotation=30,fontsize=20)
plt.yticks(fontsize=20)
f = sns.barplot(data=sell_by_day,x='event_time',y='Count')



In [ ]:
sell_by_hour=pd.DataFrame()
sell_by_hour['date' ]= oct_data_['event_time'].dt.date
sell_by_hour['hour'] = oct_data_['event_time'].dt.hour
sell_by_hour['action'] = oct_data_['event_type']
sell_by_hour= sell_by_hour.groupby('hour').size().reset_index(name='count')

In [ ]:
plt.figure(figsize=(40,20))
plt.xticks(rotation=30,fontsize=20)
plt.yticks(fontsize=20)

sns.lineplot(data=sell_by_hour,x='hour',y='count')
plt.title('Sell over time',fontsize=30)
plt.xlabel('Hour',fontsize=30)
plt.ylabel('Sell Cout', fontsize=30)



In [ ]:
def top_guns(n , dict_, dict_1 ):
    '''
    returns a list of tuples of top n number of evnts in dict and dict1 for same user.priority is max number in dict
    '''
    lst = list()
    for k in dict_.keys():
        tuple_ = (dict_.get(k,0),k)
        lst.append(tuple_)
    lst.sort()
    lst.reverse()
    
    lst1 = []
    for i in range(n):
        tuple_= lst[i]
        tuple1_=tuple((tuple_[0],dict_1.get(tuple_[1],0)))
        #print(tuple1_)
        lst1.append(tuple1_)
    
    return lst1

In [ ]:
top_purchase = pd.DataFrame()
lt = top_guns(10000,purchase_dict,cart_dict)
top_purchase = pd.DataFrame(data=lt,columns=['purchase','cart'])
tmp = pd.DataFrame(data=top_guns(10000,purchase_dict,view_dict),columns=['purchase','view'])
top_purchase['view'] = tmp['view']


In [ ]:
top_purchase.head(10000)

In [ ]:
top_purchase['scoring'] = top_purchase['purchase']*300+top_purchase['cart']*5+top_purchase['view']
top_purchase.head(5)

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
sns.scatterplot(ax=ax,data=top_purchase,x='scoring',y='purchase')
plt.xlabel('User score',fontsize=20)
plt.ylabel('Purchase count',fontsize=20)
